In [2]:
USE_GPU = False
from typing import List, Tuple, Dict, Any, Optional
import seaborn as sns
from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, recall_score, f1_score
import statsmodels.api as sm
import re
import plotly.express as px
import tensorboard
import pandas as pd
import numpy as np
import seaborn as sns
from tensorboard.backend.event_processing.event_accumulator import EventAccumulator
import matplotlib.pyplot as plt
import os
import glob
from typing import Dict
from statsmodels.tsa.filters.hp_filter import hpfilter
import re
from datetime import datetime
import plotly
import plotly.offline as pyo
import plotly.io as pio

from plotly.subplots import make_subplots
import plotly.graph_objects as go

In [3]:
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.max_rows', 500)
pd.set_option('mode.chained_assignment', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)


In [4]:
pio.renderers.default = 'notebook+pdf'
# pyo.init_notebook_mode()

In [5]:


TIME_STATS = [
    'sess_time/ended_time',
    'sess_time/session_minutes',
    'sess_time/time_cutoff',
    'sess_time/time_large',
    'sess_time/time_medium',
    'sess_time/time_small',
]

TIME_STATS_GRANULAR = [
    'ended_time', 
    'session_minutes', 
    'time_cutoff', 
    'time_large', 
    'time_medium', 
    'time_small'
]

SIZE_STATS_GRANULAR = [
    'ended_event',
    'session_size',
    'size_cutoff',
    'inc_small',
    'inc_medium',
    'inc_large'
]
    

In [44]:

def plot_vectors(df, x_name, y, title):

    fig = px.line(
        df,
        x=x_name,
        y=y,
        color='model',
        title=f'Training {title}',
    )
    

    return fig
def plot_vectors_multiple(df, y, title, show=True):

    fig = px.line(
        df,
        x='step',
        y=y,
        title=f'Training {title}',
        showlegend=show
    )
    
    # fig.show()
    return fig



def global_perc_to_end(df_matrix, session_size_bound=None):
    move_to_end_matrix = {}
    for df_name, df_path in df_matrix.items():
        print(f'Calculating percentage done for {df_name} {df_path}')
        df = pd.read_csv(df_path)
        df = df[df['exp_runs'] <= 4000]
        if session_size_bound:
            df = df[
                (df['session_size'] >= session_size_bound[0]) & (df['session_size'] <= session_size_bound[1])
            ]
       
        df = df[df['ended_event'] > 0]
        df['perc_to_end'] = df['ended_event'] / df['session_size']
        perc_to_end_summary = df.groupby(['exp_runs'])['perc_to_end'].mean().reset_index(name='perc_to_end')
        move_to_end_matrix[df_name] = perc_to_end_summary
        

    for df_name, df in move_to_end_matrix.items():
        df = df.rolling(500, center=True) \
            .mean() \
            .dropna() \
            .reset_index(drop=True)
        
        df = df[::500]
        df['model'] = df_name
        move_to_end_matrix[df_name] = df
        
    out_df = pd.concat(move_to_end_matrix.values())
    out_df = out_df.rename(columns={'exp_runs': 'episode'})
    
    return out_df


def global_perc_done(df_matrix, session_size_bound=None):
    done_matrix = {}
    for df_name, df_path in df_matrix.items():
        print(f'Calculating percentage finished for {df_name} from {df_path}')
        df = pd.read_csv(df_path)
        df = df[df['exp_runs'] <= 4000]
        if session_size_bound:
            df = df[
                (df['session_size'] >= session_size_bound[0]) & (df['session_size'] <= session_size_bound[1])]
        
        df = df[df['ended_event'] > 0]
       
       
        df['perc_done'] = df['ended_event'] == df['session_size']
        perc_done_summary = df.groupby(['exp_runs']).agg(
            done_count=('perc_done', 'sum'),
            sess_count=('perc_done', 'count')
        ) \
        .reset_index()
        perc_done_summary['perc_done'] = perc_done_summary['done_count'] / perc_done_summary['sess_count']
        done_matrix[df_name] = perc_done_summary
    
    # max_episode = min([df['exp_runs'].max() for df in done_matrix.values()])
    for df_name, df in done_matrix.items():
        df = df.rolling(250, center=True) \
            .mean() \
            .dropna() \
            .reset_index(drop=True)
        
        df = df[::250]
        df['model'] = df_name
        done_matrix[df_name] = df
        
    
    out_df = pd.concat(done_matrix.values())
    out_df = out_df.rename(columns={'exp_runs': 'episode'})
    return out_df
    
    

    

PLOTLY_COLORS = plotly.colors.DEFAULT_PLOTLY_COLORS

def plot_granular(df, model_list, showlegend=False):
    plot_list = []
    for model, color in zip(model_list, PLOTLY_COLORS):
        plot_list.append(
            go.Scatter(
                x=df['episode'],
                y=df[model],
                name=model,
                line=dict(color=color),
                showlegend=showlegend
            )
        )
    
    
    return plot_list

                

In [7]:
TB_LOGS = 'dqn_tb'
CSV_LOGS = 'rl_results/dqn_csv'

In [8]:
csv_dirs = {
    'DQN PRED CNN': 'dqn_csv/train/q2/dqn_pred_cnn.csv',
    'DQN PRED CNN BASELINE': 'dqn_csv/train/q2/dqn_pred_cnn_baseline.csv',
    'QRDQN PRED CNN': 'dqn_csv/train/q2/qrdqn_pred_cnn.csv',
    'A2C PRED CNN': 'dqn_csv/train/q2/a2c_pred_cnn.csv',
}



In [45]:
perc_to_end = global_perc_to_end(csv_dirs)
perc_finished = global_perc_done(csv_dirs)

Calculating percentage done for DQN PRED CNN dqn_csv/train/q2/dqn_pred_cnn.csv
Calculating percentage done for DQN PRED CNN BASELINE dqn_csv/train/q2/dqn_pred_cnn_baseline.csv
Calculating percentage done for QRDQN PRED CNN dqn_csv/train/q2/qrdqn_pred_cnn.csv
Calculating percentage done for A2C PRED CNN dqn_csv/train/q2/a2c_pred_cnn.csv
Calculating percentage finished for DQN PRED CNN from dqn_csv/train/q2/dqn_pred_cnn.csv
Calculating percentage finished for DQN PRED CNN BASELINE from dqn_csv/train/q2/dqn_pred_cnn_baseline.csv
Calculating percentage finished for QRDQN PRED CNN from dqn_csv/train/q2/qrdqn_pred_cnn.csv
Calculating percentage finished for A2C PRED CNN from dqn_csv/train/q2/a2c_pred_cnn.csv


In [13]:
perc_finished

,episode,done_count,sess_count,perc_done,model


In [41]:

dist_end_line = px.line(perc_to_end, x='episode', y='perc_to_end', color='model')
dist_end_line.update_layout(
    xaxis_title='Episode',
    yaxis_title='Mean Session Completion',
    width=800,
    height=500,
    margin=dict(l=50, r=50, b=50, t=50, pad=4))

dist_end_line.show()

pio.write_image(dist_end_line, 'rl_plots/q2/glob_perc_to_end.png')

In [24]:
# perc_finished = perc_finished[(perc_finished['episode'] % 25 == 0) & (perc_finished['episode'] <= 3750)]

print(perc_finished[perc_finished['model'] == 'DQN PRED CNN']['episode'].max())
print(perc_finished[perc_finished['model'] == 'QRDQN PRED CNN']['episode'].max())
print(perc_finished[perc_finished['model'] == 'A2C PRED CNN']['episode'].max())

4000
4000
4000


In [25]:
perc_finished = perc_finished.sort_values(by='episode')

In [33]:
perc_finished[perc_finished['model'] == 'DQN PRED CNN'].tail(1000)['perc_done'].mean()

0.19149057159334842

In [47]:
# perc_finished = perc_finished[(perc_finished['episode'] % 25 == 0)]
perc_finished.sort_values(by='episode')
perc_finished_line = px.line(perc_finished, x='episode', y='perc_done', color='model')
perc_finished_line.update_layout(
    xaxis_title='Episode',
    yaxis_title='Sessions Completed (%)',
    width=800,
    height=500,
    margin=dict(l=50, r=50, b=50, t=50, pad=4)
)

perc_finished_line.show()

pio.write_image(perc_finished_line, 'rl_plots/q2/glob_perc_finished.png')